In [1]:
# Cell 1: Imports
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import xgboost as xgb

from data.load_data import loadTrainingData
from plots.feature_plots import plot_roc_auc, plot_confusion_matrix
from features.create_feature_vectors import extract_features_with_expanding_window

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
# Cell 2: Load data
directories = ['../../../training_setA/', '../../../training_setB/']
max_files = None  # Adjust as needed

patient_dict = {}

for directory in directories:
    pattern = os.path.join(directory, "*.psv")
    print(f"\nLoading data from: {pattern} with max_files={max_files}")
    patient_data = loadTrainingData(
        pattern,
        max_files,
        ignore_columns=['Age', 'Gender', 'Unit1', 'Unit2', 'HospAdmTime']
    )
    patient_dict.update(patient_data)



Loading data from: ../../../training_setA/*.psv with max_files=None


Loading PSV Files: 100%|████████████████| 20336/20336 [00:17<00:00, 1185.29it/s]



Loading data from: ../../../training_setB/*.psv with max_files=None


Loading PSV Files: 100%|████████████████| 20000/20000 [00:16<00:00, 1231.17it/s]


In [ ]:
# Cell 3: Create features
feature_df = extract_features_with_expanding_window(patient_dict)
feature_df.head(10)  # Adjust as needed for a quick glance

# Drop non-feature columns if present
for col in ["patient_id", "window_size"]:
    if col in feature_df.columns:
        feature_df.drop(columns=[col], inplace=True, errors="ignore")

# Separate features and target
X = feature_df.drop(columns=["SepsisLabel"], errors="ignore")
y = feature_df["SepsisLabel"]


(1552210, 36)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.it/s]
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.7s00:00<32:39, 20.58i
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    2.0s00:01<15:58, 42.03i
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:    4.4s[00:04<16:26, 40.71
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed:    7.9s[00:07<17:12, 38.78
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   12.2s[00:12<19:52, 33.46
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:   17.4s[00:17<15:35, 42.44
[Parallel(n_jobs=-1)]: Done 866 tasks      | elapsed:   23.4s[00:23<17:14, 38.13
[Parallel(n_jobs=-1)]: Done 1136 tasks      | elapsed:   30.6s[00:30<15:13, 42.8
[Parallel(n_jobs=-1)]: Done 1442 tasks      | elapsed:   39.1s[00:39<19:57, 32.4
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed:   47.8s[00:47<14:31, 44.2
[Parallel(n_jobs=-1)]: Done 2162 tasks      | elapsed:   57.8s[00:57<15:47, 40.2
[Parallel(n_jobs=-1)]: Done 

In [ ]:
# Testing Rolling Box
for df in patient_dict.values():
    test = df
    break
    
test.head(10)

In [ ]:
feature_df.head(10)

In [ ]:
# Cell 4: Train/test split
neg_samples, pos_samples = y.value_counts()
print(f"Negative samples: {neg_samples}, Positive samples: {pos_samples}")

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

In [ ]:
# Cell 5: Train model
model = xgb.XGBClassifier(
    random_state=42,
    objective='binary:logistic',
    eval_metric="auc",
    scale_pos_weight=neg_samples / pos_samples
)
model.fit(X_train, y_train, 
          eval_set=[(X_test, y_test)],
          verbose=1)

In [ ]:
# Cell 6: Evaluate model
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]

# Plot ROC curve
plot_roc_auc(model, X_test, y_test)

# Plot confusion matrix
plot_confusion_matrix(y_test, y_pred, labels=("No Sepsis", "Sepsis"))

# Print classification metrics
print("\nClassification Report:")
print(classification_report(y_test, y_pred))